# Patroni Neural Search
This notebook applies Neural Search (Semantic Search) to the **Patroni/Postgres/Etcd** logs.
We will use the `all-MiniLM-L6-v2` model to vectorize log messages, allowing us to search for concepts like "database failure" and find logs about "terminating connection" or "lock lost".

**Steps:**
1.  Connect to OpenSearch.
2.  Ensure the Embedding Model is deployed.
3.  Create a Pipeline to vectorize the `message` field.
4.  Create the `patronidata-neural` index.
5.  Ingest the Patroni log dataset.
6.  Run Semantic Queries.

In [20]:
import requests
import json
import time
import http.client

# Increase header limit to avoid "got more than 100 headers" error
http.client._MAXHEADERS = 1000

# Configuration
HOST = 'localhost'
PORT = 19200
PROTOCOL = 'http'
AUTH = ('admin', 'OpenSearch@2024')
VERIFY_SSL = False

BASE_URL = f"{PROTOCOL}::{HOST}:{PORT}"

def run_request(method, endpoint, body=None):
    url = f"{PROTOCOL}://{HOST}:{PORT}/{endpoint}"
    headers = {"Content-Type": "application/json"}
    try:
        if method == "GET":
            resp = requests.get(url, auth=AUTH, verify=VERIFY_SSL, json=body, headers=headers)
        elif method == "POST":
            resp = requests.post(url, auth=AUTH, verify=VERIFY_SSL, json=body, headers=headers)
        elif method == "PUT":
            resp = requests.put(url, auth=AUTH, verify=VERIFY_SSL, json=body, headers=headers)
        elif method == "DELETE":
            resp = requests.delete(url, auth=AUTH, verify=VERIFY_SSL, json=body, headers=headers)
        elif method == "HEAD":
            resp = requests.head(url, auth=AUTH, verify=VERIFY_SSL, json=body, headers=headers)
        else:
            print(f"Unsupported method: {method}")
            return None
            
        return resp
    except Exception as e:
        print(f"Connection Error: {e}")
        return None

print(f"Connecting to {BASE_URL}...")
resp = run_request("GET", "")
if resp and resp.status_code == 200:
    print("✅ Connected to OpenSearch")
else:
    print("❌ Failed to connect")
    if resp: print(resp.text)

Connecting to http::localhost:19200...
✅ Connected to OpenSearch


In [16]:
# 2. Set Model ID
# User specified model ID
model_id = "RLmMzpoBDLC7DRstN6vK"

print(f"Using Model ID: {model_id}")

# Verify deployment status
resp = run_request("GET", f"_plugins/_ml/models/{model_id}")
if resp and resp.status_code == 200:
    state = resp.json().get("model_state")
    print(f"Model State: {state}")
    if state != "DEPLOYED":
        print(f"Deploying model {model_id}...")
        run_request("POST", f"_plugins/_ml/models/{model_id}/_deploy")
        time.sleep(5) # Wait a bit for deployment to initialize
else:
    print("⚠️ Model not found or inaccessible. Please ensure the model ID is correct.")

Using Model ID: RLmMzpoBDLC7DRstN6vK
Model State: DEPLOYED


In [24]:
# 3. Create Pipeline & Index
pipeline_id = "patroni-neural-pipeline"
index_name = "patronidata-neural"

# Pipeline: Maps '_raw' -> 'message_embedding'
# Note: The source data uses '_raw' for the log message, not 'message'.
pipeline_body = {
  "description": "Vectorize patroni log messages",
  "processors": [
    {
      "text_embedding": {
        "model_id": model_id,
        "field_map": {
          "_raw": "message_embedding"
        }
      }
    }
  ]
}

if model_id:
    print(f"Creating Pipeline {pipeline_id}...")
    run_request("PUT", f"_ingest/pipeline/{pipeline_id}", pipeline_body)

    # Index: KNN on 'message_embedding'
    run_request("DELETE", index_name)
    index_body = {
      "settings": {
        "index.knn": True,
        "default_pipeline": pipeline_id
      },
      "mappings": {
        "properties": {
          "timestamp": { "type": "date" },
          "hostname": { "type": "keyword" },
          "component": { "type": "keyword" },
          "level": { "type": "keyword" },
          "_raw": { "type": "text" },
          "message_embedding": {
            "type": "knn_vector",
            "dimension": 384,
            "method": {
              "name": "hnsw",
              "engine": "lucene",
              "parameters": { "m": 16, "ef_construction": 128 }
            }
          }
        }
      }
    }
    print(f"Creating Index {index_name}...")
    resp = run_request("PUT", index_name, index_body)
    print(resp.text)
else:
    print("Cannot create resources without Model ID.")

Creating Pipeline patroni-neural-pipeline...
Creating Index patronidata-neural...
{"acknowledged":true,"shards_acknowledged":true,"index":"patronidata-neural"}
{"acknowledged":true,"shards_acknowledged":true,"index":"patronidata-neural"}


In [25]:
# 4. Reindex from Real Data
# We will pull data from the existing 'patronidata' index and index it into 'patronidata-neural'.
# The 'patronidata-neural' index has a default_pipeline that will generate embeddings.

source_index = "patronidata"
dest_index = "patronidata-neural"

# Check if source exists
check = run_request("HEAD", source_index)
if check.status_code != 200:
    print(f"⚠️ Source index '{source_index}' does not exist! Please ensure you have data in it.")
else:
    print(f"Reindexing from '{source_index}' to '{dest_index}'...")

    reindex_body = {
      "source": {
        "index": source_index
      },
      "dest": {
        "index": dest_index
      }
    }

    # Use wait_for_completion=false to avoid timeouts and header issues
    resp = run_request("POST", "_reindex?wait_for_completion=false", reindex_body)

    if resp and resp.status_code == 200:
        task_id = resp.json().get("task")
        print(f"Reindex task started: {task_id}")
        
        # Poll for completion
        while True:
            task_resp = run_request("GET", f"_tasks/{task_id}")
            if task_resp and task_resp.status_code == 200:
                task_status = task_resp.json()
                completed = task_status.get("completed", False)
                
                if completed:
                    response = task_status.get("response", {})
                    total = response.get("total")
                    created = response.get("created")
                    failures = response.get("failures")
                    
                    print(f"Reindex Complete.")
                    print(f"Total processed: {total}")
                    print(f"Created/Updated: {created}")
                    
                    if failures:
                        print("⚠️ Failures occurred:")
                        print(json.dumps(failures, indent=2))
                    break
                else:
                    print("Reindexing in progress...", end="\r")
                    time.sleep(2)
            else:
                print("Failed to check task status.")
                break
        
        # Refresh to make searchable
        run_request("POST", f"{dest_index}/_refresh")
    else:
        print("Reindex Failed to start")
        if resp: print(resp.text)

Reindexing from 'patronidata' to 'patronidata-neural'...
Reindex task started: PRbPvKIxTvq8dUFkBwZE9Q:3247786


KeyboardInterrupt: 

In [26]:
# 5. Semantic Search
def neural_search(query_text, k=3):
    query = {
        "query": {
            "neural": {
                "message_embedding": {
                    "query_text": query_text,
                    "model_id": model_id,
                    "k": k
                }
            }
        },
        "_source": ["_raw", "component", "hostname"]
    }
    
    print(f"\n🔍 Query: '{query_text}'")
    resp = run_request("GET", f"{index_name}/_search", query)
    hits = resp.json().get("hits", {}).get("hits", [])
    
    for hit in hits:
        score = hit.get("_score")
        # Use _raw as the message source
        msg = hit.get("_source").get("_raw")
        # Truncate long messages
        if msg and len(msg) > 200:
            msg = msg[:200] + "..."
            
        comp = hit.get("_source").get("component", "Unknown")
        print(f"   [{score:.4f}] [{comp}] {msg}")

# Test Cases
neural_search("database went down")
neural_search("who is the new master?")
neural_search("etcd timeout")
neural_search("replication stopped")


🔍 Query: 'database went down'
   [0.3945] [Unknown] 2025-11-29 03:00:49 UTC [32]: [5554-1] user=postgres,db=postgres,app=Patroni heartbeat,client=127.0.0.1, e=00000 LOG:  statement: SELECT CASE WHEN pg_catalog.pg_is_in_recovery() THEN 0 ELSE ('x' || pg...
   [0.3940] [Unknown] 2025-11-29 03:08:49 UTC [32]: [5602-1] user=postgres,db=postgres,app=Patroni heartbeat,client=127.0.0.1, e=00000 LOG:  statement: SELECT CASE WHEN pg_catalog.pg_is_in_recovery() THEN 0 ELSE ('x' || pg...
   [0.3940] [Unknown] 2025-11-29 03:00:59 UTC [32]: [5555-1] user=postgres,db=postgres,app=Patroni heartbeat,client=127.0.0.1, e=00000 LOG:  statement: SELECT CASE WHEN pg_catalog.pg_is_in_recovery() THEN 0 ELSE ('x' || pg...

🔍 Query: 'who is the new master?'
   [0.3945] [Unknown] 2025-11-29 03:00:49 UTC [32]: [5554-1] user=postgres,db=postgres,app=Patroni heartbeat,client=127.0.0.1, e=00000 LOG:  statement: SELECT CASE WHEN pg_catalog.pg_is_in_recovery() THEN 0 ELSE ('x' || pg...
   [0.3940] [Unknown] 2025-11-

In [27]:
# 6. Debug Data Quality
print("--- Debugging Data ---")

# Check counts
count_resp = run_request("GET", f"{dest_index}/_count")
print(f"Neural Index Count: {count_resp.json().get('count')}")

# Sample Source Data
print("\n--- Sample Source Data (patronidata) ---")
resp = run_request("GET", f"{source_index}/_search?size=3")
for hit in resp.json().get("hits", {}).get("hits", []):
    print(json.dumps(hit["_source"], indent=2))

# Sample Neural Data
print("\n--- Sample Neural Data (patronidata-neural) ---")
resp = run_request("GET", f"{dest_index}/_search?size=3")
for hit in resp.json().get("hits", {}).get("hits", []):
    print(json.dumps(hit["_source"], indent=2))

--- Debugging Data ---
Neural Index Count: 32000

--- Sample Source Data (patronidata) ---
{
  "_raw": "2025-11-29 08:01:29 UTC [32]: [7358-1] user=postgres,db=postgres,app=Patroni heartbeat,client=127.0.0.1, e=00000 LOG:  statement: SELECT CASE WHEN pg_catalog.pg_is_in_recovery() THEN 0 ELSE ('x' || pg_catalog.substr(pg_catalog.pg_walfile_name(pg_catalog.pg_current_wal_lsn()), 1, 8))::bit(32)::int END, CASE WHEN pg_catalog.pg_is_in_recovery() THEN 0 ELSE pg_catalog.pg_wal_lsn_diff(pg_catalog.pg_current_wal_flush_lsn(), '0/0')::bigint END, pg_catalog.pg_wal_lsn_diff(pg_catalog.pg_last_wal_replay_lsn(), '0/0')::bigint, pg_catalog.pg_wal_lsn_diff(COALESCE(pg_catalog.pg_last_wal_receive_lsn(), '0/0'), '0/0')::bigint, pg_catalog.pg_is_in_recovery() AND pg_catalog.pg_is_wal_replay_paused(), 0, CASE WHEN latest_end_lsn IS NULL THEN NULL ELSE received_tli END, pg_catalog.pg_wal_lsn_diff(written_lsn, '0/0')::bigint, slot_name, conninfo, status, pg_catalog.current_setting('restore_command'), (S